
## Шаг 0: Проверка конфигурации

Начинаем с проверки базовых настроек и путей из core.config.

In [3]:
# Импорты и настройка окружения
import sys
from pathlib import Path

In [5]:
# Проверяем, работает ли kernel
print("🔧 Testing Jupyter kernel connection")
print(f"Python version: {__import__('sys').version}")
print(f"Current working directory: {__import__('os').getcwd()}")
print(f"Python path includes: {len(sys.path)} entries")

🔧 Testing Jupyter kernel connection
Python version: 3.13.5 (main, Aug 29 2025, 16:30:45) [GCC 11.4.0]
Current working directory: /mnt/c/Users/user/Documents/Pro/bquant/research/notebooks
Python path includes: 6 entries


In [6]:
# Добавляем путь к пакету bquant
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

Project root: /mnt/c/Users/user/Documents/Pro/bquant
Python path: /mnt/c/Users/user/Documents/Pro/bquant


In [7]:
# Импорт конфигурации
from bquant.core.config import *

print("🔧 BQuant Configuration Info")
print("=" * 40)

print(f"\n📁 Project Structure:")
print(f"  PROJECT_ROOT: {PROJECT_ROOT}")
print(f"  DATA_DIR: {DATA_DIR}")
print(f"  NOTEBOOKS_DIR: {NOTEBOOKS_DIR}")
print(f"  PROCESSED_DATA_DIR: {PROCESSED_DATA_DIR}")
print(f"  RESULTS_DIR: {RESULTS_DIR}")

print(f"\n📊 Data Validation Settings:")
for key, value in DATA_VALIDATION.items():
    print(f"  {key}: {value}")

print(f"\n⏱️ Supported Timeframes ({len(SUPPORTED_TIMEFRAMES)} total):")
for tf, desc in list(SUPPORTED_TIMEFRAMES.items())[:10]:  # Показываем первые 10
    print(f"  {tf}: {desc}")
if len(SUPPORTED_TIMEFRAMES) > 10:
    print(f"  ... и еще {len(SUPPORTED_TIMEFRAMES) - 10} таймфреймов")

🔧 BQuant Configuration Info

📁 Project Structure:
  PROJECT_ROOT: /mnt/c/Users/user/Documents/Pro/bquant
  DATA_DIR: /mnt/c/Users/user/Documents/Pro/bquant/data
  NOTEBOOKS_DIR: /mnt/c/Users/user/Documents/Pro/bquant/research/notebooks
  PROCESSED_DATA_DIR: /mnt/c/Users/user/Documents/Pro/bquant/data/processed
  RESULTS_DIR: /mnt/c/Users/user/Documents/Pro/bquant/results

📊 Data Validation Settings:
  required_columns: ['open', 'high', 'low', 'close']
  optional_columns: ['volume']
  min_records: 100
  max_missing_ratio: 0.1
  outlier_threshold: 3.0

⏱️ Supported Timeframes (27 total):
  1m: 1 minute
  2m: 2 minutes
  3m: 3 minutes
  4m: 4 minutes
  5m: 5 minutes
  6m: 6 minutes
  10m: 10 minutes
  12m: 12 minutes
  15m: 15 minutes
  20m: 20 minutes
  ... и еще 17 таймфреймов


In [8]:
# Проверка существования директорий
print("📂 Directory Status:")
directories = {
    'PROJECT_ROOT': PROJECT_ROOT,
    'DATA_DIR': DATA_DIR, 
    'NOTEBOOKS_DIR': NOTEBOOKS_DIR,
    'PROCESSED_DATA_DIR': PROCESSED_DATA_DIR,
    'RESULTS_DIR': RESULTS_DIR
}

for name, path in directories.items():
    status = "✅ Exists" if path.exists() else "❌ Missing"
    print(f"  {name}: {status}")
    if path.exists() and path.is_dir():
        files_count = len(list(path.glob('*')))
        print(f"    Files/dirs: {files_count}")

📂 Directory Status:
  PROJECT_ROOT: ✅ Exists
    Files/dirs: 25
  DATA_DIR: ✅ Exists
    Files/dirs: 2
  NOTEBOOKS_DIR: ✅ Exists
    Files/dirs: 2
  PROCESSED_DATA_DIR: ✅ Exists
    Files/dirs: 0
  RESULTS_DIR: ✅ Exists
    Files/dirs: 1


In [9]:
# Тестирование функций конфигурации
print("🧪 Testing Config Functions:")
print("=" * 30)

# Тест get_data_path
print("\n📄 get_data_path() examples:")
test_cases = [
    ('XAUUSD', '1h', 'tradingview', 'oanda'),
    ('EURUSD', '1d', 'tradingview', 'forexcom'),
    ('XAUUSD', '15m', 'metatrader', 'default')
]

for symbol, tf, source, provider in test_cases:
    try:
        path = get_data_path(symbol, tf, source, provider)
        exists = "✅" if path.exists() else "❌"
        print(f"  {symbol}_{tf}_{source}_{provider}: {exists} {path.name}")
    except Exception as e:
        print(f"  {symbol}_{tf}_{source}_{provider}: ❌ Error: {e}")

# Тест validate_timeframe
print("\n⏱️ validate_timeframe() examples:")
test_timeframes = ['1h', '1d', '15m', '5m', 'invalid']
for tf in test_timeframes:
    try:
        result = validate_timeframe(tf)
        print(f"  {tf}: ✅ Valid -> {result}")
    except ValueError as e:
        print(f"  {tf}: ❌ Invalid -> {str(e)[:50]}...")

🧪 Testing Config Functions:

📄 get_data_path() examples:
  XAUUSD_1h_tradingview_oanda: ❌ OANDA_XAUUSD, 60.csv
  EURUSD_1d_tradingview_forexcom: ❌ FOREXCOM_EURUSD, 1D.csv
  XAUUSD_15m_metatrader_default: ❌ XAUUSDM15.csv

⏱️ validate_timeframe() examples:
  1h: ✅ Valid -> 1h
  1d: ✅ Valid -> 1d
  15m: ✅ Valid -> 15m
  5m: ✅ Valid -> 5m
  invalid: ❌ Invalid -> Unsupported timeframe: invalid. Supported: ['1m', ...


## Шаг 1: Тестирование новых функций управления директориями

Проверим работу добавленных функций для управления путями директорий.

# 01. BQuant Data Module Testing

Тестирование модулей данных пакета bquant с пошаговой проверкой функциональности.

In [12]:
# Импорт новых функций управления директориями
from bquant.core.config import (
    get_data_dir, set_data_dir, 
    get_results_dir, set_results_dir,
    get_notebooks_dir, set_notebooks_dir,
    get_processed_data_dir, set_processed_data_dir,
    reset_directories_to_defaults,
    get_directory_status
)

print("🧪 Testing Directory Management Functions")
print("=" * 45)

# Проверяем текущее состояние директорий
print("\n📂 Current Directory Status:")
status = get_directory_status()

for dir_name, info in status.items():
    custom_flag = "CUSTOM" if info['is_custom'] else "DEFAULT"
    exists_flag = "EXISTS" if info['exists'] else "NOT_EXISTS"
    print(f"  {dir_name}: STATUS - {custom_flag} | {exists_flag}")
    print(f"    Path: {info['current']}")
    if info['is_custom']:
        print(f"    Default: {info['default']}")

🧪 Testing Directory Management Functions

📂 Current Directory Status:
  data_dir: STATUS - DEFAULT | EXISTS
    Path: /mnt/c/Users/user/Documents/Pro/bquant/data
  results_dir: STATUS - DEFAULT | EXISTS
    Path: /mnt/c/Users/user/Documents/Pro/bquant/results
  notebooks_dir: STATUS - DEFAULT | EXISTS
    Path: /mnt/c/Users/user/Documents/Pro/bquant/research/notebooks
  processed_data_dir: STATUS - DEFAULT | EXISTS
    Path: /mnt/c/Users/user/Documents/Pro/bquant/data/processed


In [13]:
# Тестируем установку кастомного пути для data - используем актуальную папку data/row
print("\n🔧 Testing Custom Data Directory:")
print(f"Default data dir: {get_data_dir()}")

# Устанавливаем путь к актуальным данным
raw_data_path = PROJECT_ROOT / "data" / "row"
print(f"Setting data dir to: {raw_data_path}")

try:
    set_data_dir(raw_data_path)
    print(f"✅ New data dir: {get_data_dir()}")
    
    # Проверяем, что директория создалась
    if get_data_dir().exists():
        print("✅ Directory exists")
        # Смотрим содержимое
        files = list(get_data_dir().glob("*"))
        print(f"📁 Contains {len(files)} files/dirs:")
        for f in files[:5]:  # Показываем первые 5
            print(f"    {f.name}")
        if len(files) > 5:
            print(f"    ... and {len(files) - 5} more")
    else:
        print("❌ Directory doesn't exist yet")
        
except Exception as e:
    print(f"❌ Error: {e}")


🔧 Testing Custom Data Directory:
Default data dir: /mnt/c/Users/user/Documents/Pro/bquant/data
Setting data dir to: /mnt/c/Users/user/Documents/Pro/bquant/data/row
✅ New data dir: /mnt/c/Users/user/Documents/Pro/bquant/data/row
✅ Directory exists
📁 Contains 3 files/dirs:
    alldata
    OANDA_XAUUSD, 1D.csv
    OANDA_XAUUSD, 60.csv


In [14]:
# Проверяем статус после изменений
print("\n📊 Directory Status After Changes:")
status = get_directory_status()

for dir_name, info in status.items():
    custom_flag = "🔧 CUSTOM" if info['is_custom'] else "📦 DEFAULT"
    exists_flag = "✅" if info['exists'] else "❌"
    print(f"  {dir_name}: {custom_flag} {exists_flag}")
    print(f"    Current: {info['current']}")
    if info['is_custom'] and info['default'] != info['current']:
        print(f"    Default: {info['default']}")


📊 Directory Status After Changes:
  data_dir: 🔧 CUSTOM ✅
    Current: /mnt/c/Users/user/Documents/Pro/bquant/data/row
    Default: /mnt/c/Users/user/Documents/Pro/bquant/data
  results_dir: 📦 DEFAULT ✅
    Current: /mnt/c/Users/user/Documents/Pro/bquant/results
  notebooks_dir: 📦 DEFAULT ✅
    Current: /mnt/c/Users/user/Documents/Pro/bquant/research/notebooks
  processed_data_dir: 📦 DEFAULT ✅
    Current: /mnt/c/Users/user/Documents/Pro/bquant/data/processed


In [15]:
# Тестируем функцию сброса к умолчаниям
print("\n🔄 Testing Reset to Defaults:")

# Сначала покажем что у нас кастомные настройки
print("Before reset:")
print(f"  Data dir is custom: {get_directory_status()['data_dir']['is_custom']}")
print(f"  Current data dir: {get_data_dir()}")

# Сбрасываем
reset_directories_to_defaults()

print("\nAfter reset:")
print(f"  Data dir is custom: {get_directory_status()['data_dir']['is_custom']}")
print(f"  Current data dir: {get_data_dir()}")

# Проверяем полный статус
print("\n📋 Final Status Summary:")
status = get_directory_status()
all_default = all(not info['is_custom'] for info in status.values())
print(f"All directories using defaults: {'✅ Yes' if all_default else '❌ No'}")


🔄 Testing Reset to Defaults:
Before reset:
  Data dir is custom: True
  Current data dir: /mnt/c/Users/user/Documents/Pro/bquant/data/row

After reset:
  Data dir is custom: False
  Current data dir: /mnt/c/Users/user/Documents/Pro/bquant/data

📋 Final Status Summary:
All directories using defaults: ✅ Yes


In [20]:
# Финальный тест - установим data/row как рабочую директорию и оставим
print("\n🎯 Final Setup - Setting data/row as working data directory:")

# Устанавливаем актуальную папку с данными
raw_data_path = PROJECT_ROOT / "data" / "row"
set_data_dir(raw_data_path)

print(f"✅ Active data directory: {get_data_dir()}")
print(f"✅ Directory exists: {get_data_dir().exists()}")

if get_data_dir().exists():
    # Показываем что есть в папке
    csv_files = list(get_data_dir().glob("*.csv"))
    print(f"📄 Found {len(csv_files)} CSV files:")
    for csv_file in csv_files[:5]:  # Первые 5
        print(f"    {csv_file.name}")
    if len(csv_files) > 5:
        print(f"    ... and {len(csv_files) - 3} more CSV files")

print("\n🔧 Ready for data loading tests!")


🎯 Final Setup - Setting data/row as working data directory:
✅ Active data directory: /mnt/c/Users/user/Documents/Pro/bquant/data/row
✅ Directory exists: True
📄 Found 4 CSV files:
    OANDA_XAUUSD, 15.csv
    OANDA_XAUUSD, 1D.csv
    OANDA_XAUUSD, 60.csv
    XAUUSDH1.csv

🔧 Ready for data loading tests!
